In [1]:
%matplotlib inline
import pandas
import matplotlib
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


# Average window_stride elements together to form a single row
window_stride = 12

sample_hours = window_stride / 12.0
print("Sample Hours: %f" % sample_hours)

# Number of future samples to mean for prediction
prediction_window = int(24 / sample_hours)
print("Prediction Window: %d" % prediction_window)

# Length of the windowed sequence
sequence_length = int(7*24 / sample_hours)
print("Sequence Length: %d" % sequence_length)

# Number of features we take from the data
input_features = 9
num_features = input_features
num_inputs = input_features

# Number of things we are doing regression to predict
num_outputs = 4

# Input Features
columns = ['hour', 'temp', 'windspd', 'winddir', 'no', 'no2', 'nox', 'o3', 'epoch']

# Read the data
df = pandas.read_csv('d00.csv')

# Drop useless columns
df = df.drop(['AQS_Code', 'Latitude', 'Longitude', 'day'], axis=1)

# Unprocessed dataset
nd = df[columns].values

# Windowed dataset
nd_window = np.zeros((int(nd.shape[0] / window_stride), num_inputs))

row = 0
while row < nd.shape[0] - window_stride:
    for i in range(0, input_features):
        nd_window[int(row/window_stride)][i] = np.mean(nd[row:row+window_stride,i])
    row += window_stride
    
scaler = MinMaxScaler()
scaler.fit(nd_window)
nd_window = scaler.transform(nd_window)


# Create sequences
data = []
labels = []


rows = deque(maxlen=sequence_length)

for idx, r in enumerate(nd_window):

    rows.append([a for a in r])
    
    # We need the entire sequence filled to make a prediction about the future mean
    if len(rows) < sequence_length:
        continue
    
    # Since we are predicting the mean, make sure we do not go out of bounds in the future
    if idx+1 + prediction_window > nd_window.shape[0]:
        break
        
    data.append(rows.copy())
        
    # We are predicting the future mean values
    u_24_no = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 4] )
    u_24_no2 = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 5] )
    u_24_nox = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 6] )
    u_24_o3 = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 7] )
    
    labels.append([u_24_no, u_24_no2, u_24_nox, u_24_o3])

data = np.array(data)
labels = np.array(labels)

Sample Hours: 1.000000
Prediction Window: 24
Sequence Length: 168


In [2]:
from keras.models import Model
from keras import backend as K
from keras.layers import Dense, LSTM, Input, Flatten
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard

def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

layer_input = Input(shape=(sequence_length, input_features), name='inputs')

# For some reason putting some extra dimensions before an LSTM works wonders
dense_1 = Dense(128, input_dim=(sequence_length, input_features))(layer_input)

layer_lstm = LSTM(64, return_sequences=True, dropout=0.5)(dense_1)
layer_flatten = Flatten()(layer_lstm)

layer_output = Dense(num_outputs, activation='linear', name='outputs')(layer_flatten)

model = Model(inputs=[layer_input], outputs=[layer_output])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[r2])
model.summary()

def sched(epoch, lr):
    new_lr = 0.001 * (0.95 ** epoch)
    print("Epoch(%d) LR: %f" % (epoch+1, new_lr))
    return new_lr

lr_decay = LearningRateScheduler(schedule=sched) 

filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_r2', verbose=1, save_best_only=True, mode='max')

tensorboard = TensorBoard(log_dir='./tb', histogram_freq=0, batch_size=128, write_graph=True, write_grads=False)

model.fit(x=data, y=labels, batch_size=128, epochs=100, validation_split=0.2, verbose=True, callbacks=[lr_decay, checkpoint, tensorboard])

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 168, 9)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 168, 128)          1280      
_________________________________________________________________
lstm_1 (LSTM)                (None, 168, 64)           49408     
_________________________________________________________________
flatten_1 (Flatten)          (None, 10752)             0         
_________________________________________________________________
outputs (Dense)              (None, 4)                 43012     
Total params: 93,700
Trainable params: 93,700
Non-trainable params: 0
_________________________________________________________________
Train on 6513 samples, validate on 1629 samples
Epoch 1/100
Epoch(1) LR: 0.001000
6513/6513 [==============================] - 15s 2ms/ste

6513/6513 [==============================] - 14s 2ms/step - loss: 9.0113e-04 - r2: 0.9092 - val_loss: 0.0094 - val_r2: 0.4878

Epoch 00035: val_r2 did not improve from 0.71662
Epoch 36/100
Epoch(36) LR: 0.000166
6513/6513 [==============================] - 14s 2ms/step - loss: 8.9170e-04 - r2: 0.9103 - val_loss: 0.0096 - val_r2: 0.4825

Epoch 00036: val_r2 did not improve from 0.71662
Epoch 37/100
Epoch(37) LR: 0.000158
6513/6513 [==============================] - 14s 2ms/step - loss: 8.9835e-04 - r2: 0.9096 - val_loss: 0.0097 - val_r2: 0.4902

Epoch 00037: val_r2 did not improve from 0.71662
Epoch 38/100
Epoch(38) LR: 0.000150
6513/6513 [==============================] - 14s 2ms/step - loss: 8.9604e-04 - r2: 0.9097 - val_loss: 0.0096 - val_r2: 0.4827

Epoch 00038: val_r2 did not improve from 0.71662
Epoch 39/100
Epoch(39) LR: 0.000142
6513/6513 [==============================] - 14s 2ms/step - loss: 8.6323e-04 - r2: 0.9132 - val_loss: 0.0099 - val_r2: 0.4711

Epoch 00039: val_r2 did n

6513/6513 [==============================] - 14s 2ms/step - loss: 7.4526e-04 - r2: 0.9250 - val_loss: 0.0109 - val_r2: 0.4320

Epoch 00074: val_r2 did not improve from 0.71662
Epoch 75/100
Epoch(75) LR: 0.000022
6513/6513 [==============================] - 14s 2ms/step - loss: 7.4211e-04 - r2: 0.9252 - val_loss: 0.0113 - val_r2: 0.4207

Epoch 00075: val_r2 did not improve from 0.71662
Epoch 76/100
Epoch(76) LR: 0.000021
6513/6513 [==============================] - 14s 2ms/step - loss: 7.2418e-04 - r2: 0.9272 - val_loss: 0.0113 - val_r2: 0.4208

Epoch 00076: val_r2 did not improve from 0.71662
Epoch 77/100
Epoch(77) LR: 0.000020
6513/6513 [==============================] - 14s 2ms/step - loss: 7.3135e-04 - r2: 0.9264 - val_loss: 0.0115 - val_r2: 0.4103

Epoch 00077: val_r2 did not improve from 0.71662
Epoch 78/100
Epoch(78) LR: 0.000019
6513/6513 [==============================] - 14s 2ms/step - loss: 7.2749e-04 - r2: 0.9268 - val_loss: 0.0113 - val_r2: 0.4236

Epoch 00078: val_r2 did n

In [ ]:
model.load_weights("weights.best.hdf5")

plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['font.size'] = 16

for seq in range(0, data.shape[0] - sequence_length):
    
    lookup = {'no': (0, 0), 'no2':(0, 1), 'nox':(1, 0), 'o3':(1, 1)}

    pred = model.predict(data[seq].reshape(1, sequence_length, num_features))[0]
    fig, ax = plt.subplots(2, 2)

    for idx,f in enumerate([(4, 'no'), (5, 'no2'), (6, 'nox'), (7, 'o3')]):
    
        feature_index, feature_name = f
        
        X = []
        Y_actual = []

        for i in range(0, sequence_length + int(24*(1/sample_hours))):
            X.append(seq+i)
            Y_actual.append(data[seq+i][-1][feature_index])

        Y_actual = np.array(Y_actual)
        
        predicted_mean = pred[feature_index - 4]
        actual_mean = np.mean(Y_actual[sequence_length:])
        rolling_mean = np.mean(Y_actual[:sequence_length])
        rolling_std = np.std(Y_actual[:sequence_length])
                
        Y_pred = Y_actual.copy()
        Y_pred[sequence_length:] = predicted_mean
        Y_pred[:sequence_length] = np.nan

        Y_actual_mean = Y_actual.copy()
        Y_actual_mean[sequence_length:] = actual_mean
        Y_actual_mean[:sequence_length] = np.nan
        
        Y_rolling_mean = Y_actual.copy()
        Y_rolling_mean[:sequence_length] = rolling_mean
        Y_rolling_mean[sequence_length:] = np.nan
        
        Y_rolling_std_upper = Y_actual.copy()
        Y_rolling_std_upper[:sequence_length] = rolling_mean + rolling_std
        Y_rolling_std_upper[sequence_length:] = np.nan
        
        Y_rolling_std_lower = Y_actual.copy()
        Y_rolling_std_lower[:sequence_length] = rolling_mean - rolling_std
        Y_rolling_std_lower[sequence_length:] = np.nan   
        
        subplot = ax[lookup[feature_name][0]][lookup[feature_name][1]]

        subplot.plot(X, Y_actual, color='black', linewidth=4.0)
        subplot.plot(X, Y_actual_mean, color='green', linewidth=4.0)
        subplot.plot(X, Y_pred, color='purple', linewidth=4.0)
        subplot.plot(X, Y_rolling_mean, color='green', linewidth=4.0)
        subplot.plot(X, Y_rolling_std_upper, color='orange', linewidth=4.0)
        subplot.plot(X, Y_rolling_std_lower, color='orange', linewidth=4.0)
        
        subplot.grid()
        
        subplot.set_title("%s 24 hour mean prediction" % (feature_name,))
        
        subplot.set_xlabel("Hours")
        subplot.set_ylabel("Scaled Concentration")
    
    fig.legend(['Actual Continuous', 'Actual Mean', 'Predicted Mean', 'Rolling Mean', 'Standard Deviation'])
    fig.tight_layout()

    plt.savefig('charts/%.05d.png' % seq)
    #plt.show()
    plt.close()

    print("Rendered %d" % seq)

Rendered 0
Rendered 1
Rendered 2
Rendered 3
Rendered 4
Rendered 5
Rendered 6
Rendered 7
Rendered 8
Rendered 9
Rendered 10
Rendered 11
Rendered 12
Rendered 13
Rendered 14
Rendered 15
Rendered 16
Rendered 17
Rendered 18
Rendered 19
Rendered 20
Rendered 21
Rendered 22
Rendered 23
Rendered 24
Rendered 25
Rendered 26
Rendered 27
Rendered 28
Rendered 29
Rendered 30
Rendered 31
Rendered 32
Rendered 33
Rendered 34
Rendered 35
Rendered 36
Rendered 37
Rendered 38
Rendered 39
Rendered 40
Rendered 41
Rendered 42
Rendered 43
Rendered 44
Rendered 45
Rendered 46
Rendered 47
Rendered 48
Rendered 49
Rendered 50
Rendered 51
Rendered 52
Rendered 53
Rendered 54
Rendered 55
Rendered 56
Rendered 57
Rendered 58
Rendered 59
Rendered 60
Rendered 61
Rendered 62
Rendered 63
Rendered 64
Rendered 65
Rendered 66
Rendered 67
Rendered 68
Rendered 69
Rendered 70
Rendered 71
Rendered 72
Rendered 73
Rendered 74
Rendered 75
Rendered 76
Rendered 77
Rendered 78
Rendered 79
Rendered 80
Rendered 81
Rendered 82
Rendered 83
Re

Rendered 639
Rendered 640
Rendered 641
Rendered 642
Rendered 643
Rendered 644
Rendered 645
Rendered 646
Rendered 647
Rendered 648
Rendered 649
Rendered 650
Rendered 651
Rendered 652
Rendered 653
Rendered 654
Rendered 655
Rendered 656
Rendered 657
Rendered 658
Rendered 659
Rendered 660
Rendered 661
Rendered 662
Rendered 663
Rendered 664
Rendered 665
Rendered 666
Rendered 667
Rendered 668
Rendered 669
Rendered 670
Rendered 671
Rendered 672
Rendered 673
Rendered 674
Rendered 675
Rendered 676
Rendered 677
Rendered 678
Rendered 679
Rendered 680
Rendered 681
Rendered 682
Rendered 683
Rendered 684
Rendered 685
Rendered 686
Rendered 687
Rendered 688
Rendered 689
Rendered 690
Rendered 691
Rendered 692
Rendered 693
Rendered 694
Rendered 695
Rendered 696
Rendered 697
Rendered 698
Rendered 699
Rendered 700
Rendered 701
Rendered 702
Rendered 703
Rendered 704
Rendered 705
Rendered 706
Rendered 707
Rendered 708
Rendered 709
Rendered 710
Rendered 711
Rendered 712
Rendered 713
Rendered 714
Rendered 715

Rendered 1250
Rendered 1251
Rendered 1252
Rendered 1253
Rendered 1254
Rendered 1255
Rendered 1256
Rendered 1257
Rendered 1258
Rendered 1259
Rendered 1260
Rendered 1261
Rendered 1262
Rendered 1263
Rendered 1264
Rendered 1265
Rendered 1266
Rendered 1267
Rendered 1268
Rendered 1269
Rendered 1270
Rendered 1271
Rendered 1272
Rendered 1273
Rendered 1274
Rendered 1275
Rendered 1276
Rendered 1277
Rendered 1278
Rendered 1279
Rendered 1280
Rendered 1281
Rendered 1282
Rendered 1283
Rendered 1284
Rendered 1285
Rendered 1286
Rendered 1287
Rendered 1288
Rendered 1289
Rendered 1290
Rendered 1291
Rendered 1292
Rendered 1293
Rendered 1294
Rendered 1295
Rendered 1296
Rendered 1297
Rendered 1298
Rendered 1299
Rendered 1300
Rendered 1301
Rendered 1302
Rendered 1303
Rendered 1304
Rendered 1305
Rendered 1306
Rendered 1307
Rendered 1308
Rendered 1309
Rendered 1310
Rendered 1311
Rendered 1312
Rendered 1313
Rendered 1314
Rendered 1315
Rendered 1316
Rendered 1317
Rendered 1318
Rendered 1319
Rendered 1320
Render

Rendered 1836
Rendered 1837
Rendered 1838
Rendered 1839
Rendered 1840
Rendered 1841
Rendered 1842
Rendered 1843
Rendered 1844
Rendered 1845
Rendered 1846
Rendered 1847
Rendered 1848
Rendered 1849
Rendered 1850
Rendered 1851
Rendered 1852
Rendered 1853
Rendered 1854
Rendered 1855
Rendered 1856
Rendered 1857
Rendered 1858
Rendered 1859
Rendered 1860
Rendered 1861
Rendered 1862
Rendered 1863
Rendered 1864
Rendered 1865
Rendered 1866
Rendered 1867
Rendered 1868
Rendered 1869
Rendered 1870
Rendered 1871
Rendered 1872
Rendered 1873
Rendered 1874
Rendered 1875
Rendered 1876
Rendered 1877
Rendered 1878
Rendered 1879
Rendered 1880
Rendered 1881
Rendered 1882
Rendered 1883
Rendered 1884
Rendered 1885
Rendered 1886
Rendered 1887
Rendered 1888
Rendered 1889
Rendered 1890
Rendered 1891
Rendered 1892
Rendered 1893
Rendered 1894
Rendered 1895
Rendered 1896
Rendered 1897
Rendered 1898
Rendered 1899
Rendered 1900
Rendered 1901
Rendered 1902
Rendered 1903
Rendered 1904
Rendered 1905
Rendered 1906
Render

Rendered 2422
Rendered 2423
Rendered 2424
Rendered 2425
Rendered 2426
Rendered 2427
Rendered 2428
Rendered 2429
Rendered 2430
Rendered 2431
Rendered 2432
Rendered 2433
Rendered 2434
Rendered 2435
Rendered 2436
Rendered 2437
Rendered 2438
Rendered 2439
Rendered 2440
Rendered 2441
Rendered 2442
Rendered 2443
Rendered 2444
Rendered 2445
Rendered 2446
Rendered 2447
Rendered 2448
Rendered 2449
Rendered 2450
Rendered 2451
Rendered 2452
Rendered 2453
Rendered 2454
Rendered 2455
Rendered 2456
Rendered 2457
Rendered 2458
Rendered 2459
Rendered 2460
Rendered 2461
Rendered 2462
Rendered 2463
Rendered 2464
Rendered 2465
Rendered 2466
Rendered 2467
Rendered 2468
Rendered 2469
Rendered 2470
Rendered 2471
Rendered 2472
Rendered 2473
Rendered 2474
Rendered 2475
Rendered 2476
Rendered 2477
Rendered 2478
Rendered 2479
Rendered 2480
Rendered 2481
Rendered 2482
Rendered 2483
Rendered 2484
Rendered 2485
Rendered 2486
Rendered 2487
Rendered 2488
Rendered 2489
Rendered 2490
Rendered 2491
Rendered 2492
Render

Rendered 3008
Rendered 3009
Rendered 3010
Rendered 3011
Rendered 3012
Rendered 3013
Rendered 3014
Rendered 3015
Rendered 3016
Rendered 3017
Rendered 3018
Rendered 3019
Rendered 3020
Rendered 3021
Rendered 3022
Rendered 3023
Rendered 3024
Rendered 3025
Rendered 3026
Rendered 3027
Rendered 3028
Rendered 3029
Rendered 3030
Rendered 3031
Rendered 3032
Rendered 3033
Rendered 3034
Rendered 3035
Rendered 3036
Rendered 3037
Rendered 3038
Rendered 3039
Rendered 3040
Rendered 3041
Rendered 3042
Rendered 3043
Rendered 3044
Rendered 3045
Rendered 3046
Rendered 3047
Rendered 3048
Rendered 3049
Rendered 3050
Rendered 3051
Rendered 3052
Rendered 3053
Rendered 3054
Rendered 3055
Rendered 3056
Rendered 3057
Rendered 3058
Rendered 3059
Rendered 3060
Rendered 3061
Rendered 3062
Rendered 3063
Rendered 3064
Rendered 3065
Rendered 3066
Rendered 3067
Rendered 3068
Rendered 3069
Rendered 3070
Rendered 3071
Rendered 3072
Rendered 3073
Rendered 3074
Rendered 3075
Rendered 3076
Rendered 3077
Rendered 3078
Render

Rendered 3594
Rendered 3595
Rendered 3596
Rendered 3597
Rendered 3598
Rendered 3599
Rendered 3600
Rendered 3601
Rendered 3602
Rendered 3603
Rendered 3604
Rendered 3605
Rendered 3606
Rendered 3607
Rendered 3608
Rendered 3609
Rendered 3610
Rendered 3611
Rendered 3612
Rendered 3613
Rendered 3614
Rendered 3615
Rendered 3616
Rendered 3617
Rendered 3618
Rendered 3619
Rendered 3620
Rendered 3621
Rendered 3622
Rendered 3623
Rendered 3624
Rendered 3625
Rendered 3626
Rendered 3627
Rendered 3628
Rendered 3629
Rendered 3630
Rendered 3631
Rendered 3632
Rendered 3633
Rendered 3634
Rendered 3635
Rendered 3636
Rendered 3637
Rendered 3638
Rendered 3639
Rendered 3640
Rendered 3641
Rendered 3642
Rendered 3643
Rendered 3644
Rendered 3645
Rendered 3646
Rendered 3647
Rendered 3648
Rendered 3649
Rendered 3650
Rendered 3651
Rendered 3652
Rendered 3653
Rendered 3654
Rendered 3655
Rendered 3656
Rendered 3657
Rendered 3658
Rendered 3659
Rendered 3660
Rendered 3661
Rendered 3662
Rendered 3663
Rendered 3664
Render